<a href="https://colab.research.google.com/github/sharanyazx/Kovai.co-Assessment/blob/main/Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
from statsmodels.tsa.api import VAR
import numpy as np
import warnings

# Suppress specific pandas warnings
warnings.filterwarnings('ignore', message='Could not infer format')
warnings.filterwarnings('ignore', message='A date index has been provided')

try:
    # Step 1: Load the dataset
    print("Loading dataset...")
    df = pd.read_csv(
        '/content/Daily_Public_Transport_Passenger_Journeys_by_Service_Type_20250527.csv',
        header=None,
        names=[
            'date',
            'Local_Route',
            'Light_Rail',
            'Peak_Service',
            'Rapid_Route',
            'School',
            'Other'
        ]
    )
    print(f"Dataset loaded with shape: {df.shape}")
    print("Preview of the data:")
    print(df.head())

    # Step 2: Parse dates with multiple format attempts
    print("\nParsing the 'date' column...")
    date_strings = df['date'].astype(str)

    date_formats = [
        '%d/%m/%Y', '%d-%m-%Y', '%Y-%m-%d', '%m/%d/%Y',
        '%d/%m/%y', '%d-%m-%y', '%y-%m-%d', '%m/%d/%y'
    ]

    parsed_dates = None
    for fmt in date_formats:
        try:
            parsed_dates = pd.to_datetime(date_strings, format=fmt, errors='raise')
            print(f"Dates successfully parsed with format: {fmt}")
            break
        except Exception:
            continue

    if parsed_dates is None:
        print("Failed to parse with predefined formats, trying automatic parsing...")
        parsed_dates = pd.to_datetime(date_strings, errors='coerce', dayfirst=True)

    df['date'] = parsed_dates

    # Check for any unparsed dates and drop them
    missing_dates = df['date'].isna().sum()
    if missing_dates > 0:
        print(f"Warning: {missing_dates} dates could not be parsed and will be removed.")
        df.dropna(subset=['date'], inplace=True)

    # Step 3: Convert selected columns to numeric types
    print("\nConverting data columns to numeric types...")
    numeric_columns = ['Local_Route', 'Light_Rail', 'Peak_Service', 'Rapid_Route', 'School']

    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Set date as index and sort
    df.set_index('date', inplace=True)
    df = df[numeric_columns].sort_index()

    # Remove duplicate dates, keep first occurrence
    df = df[~df.index.duplicated(keep='first')]

    print("Data types after conversion:")
    print(df.dtypes)
    print("\nCount of missing values per column:")
    print(df.isnull().sum())

    # Step 4: Data cleaning - drop rows with missing or infinite values
    print("\nCleaning data by removing missing and infinite values...")
    initial_shape = df.shape
    df_clean = df.replace([np.inf, -np.inf], np.nan).dropna()
    cleaned_shape = df_clean.shape
    removed_rows = initial_shape[0] - cleaned_shape[0]

    print(f"Initial data size: {initial_shape}")
    print(f"Cleaned data size: {cleaned_shape}")
    print(f"Rows removed: {removed_rows}")

    if cleaned_shape[0] < 10:
        raise ValueError(f"Not enough valid data after cleaning. Only {cleaned_shape[0]} rows remain.")

    # Step 5: Display summary statistics
    print("\nSummary of cleaned dataset:")
    print(f"Date range: {df_clean.index.min().strftime('%Y-%m-%d')} to {df_clean.index.max().strftime('%Y-%m-%d')}")
    print(f"Total records: {cleaned_shape[0]}")
    print("Sample data:")
    print(df_clean.head())
    print("\nDescriptive statistics:")
    print(df_clean.describe().round(2))

    # Step 6: Fit VAR model
    print("\n" + "--"*50)
    print("Fitting VAR model")
    print("--"*50)

    var_model = VAR(df_clean.values)

    max_lag_candidates = min(12, cleaned_shape[0] // 5)
    print(f"Considering up to {max_lag_candidates} lags for the model.")

    try:
        results = var_model.fit(maxlags=max_lag_candidates, ic='aic')
        selection_method = 'AIC'
    except Exception:
        print("AIC-based lag selection failed. Trying BIC...")
        try:
            results = var_model.fit(maxlags=max_lag_candidates, ic='bic')
            selection_method = 'BIC'
        except Exception:
            print("Automatic lag selection failed. Using a fixed lag of 2.")
            results = var_model.fit(maxlags=2)
            selection_method = 'Fixed'

    print("Model fitted successfully!")
    print(f"Lag order selected by {selection_method}: {results.k_ar}")
    print(f"Number of observations used: {results.nobs}")
    print(f"AIC: {results.aic:.2f}")
    print(f"BIC: {results.bic:.2f}")

    # Step 7: Forecasting future values
    print("\n" + "--"*50)
    print("Generating 7-day forecast")
    print("--"*50)

    lag_order = results.k_ar
    input_for_forecast = df_clean.values[-lag_order:]

    print(f"Using last {lag_order} observations for forecast input.")

    forecast_horizon = 7
    forecasted_values = results.forecast(y=input_for_forecast, steps=forecast_horizon)

    last_date = df_clean.index[-1]
    forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=forecast_horizon, freq='D')

    forecast_df = pd.DataFrame(forecasted_values, index=forecast_dates, columns=df_clean.columns)
    forecast_df = forecast_df.clip(lower=0)  # No negative forecasts assumed

    # Step 8: Present forecast results
    print("\n" + "--"*60)
    print("Forecasted values for next 7 days:")
    print("--"*60)

    rounded_forecast = forecast_df.round(1)
    print(rounded_forecast.to_string())

    # Display summary statistics of forecast
    print("\n" + "--"*50)
    print("Forecast Summary")
    print("--"*50)

    daily_totals = forecast_df.sum(axis=1)
    print("Total forecasted passengers per day:")
    for date, total in daily_totals.items():
        print(f"  {date.strftime('%Y-%m-%d')}: {total:.1f}")

    print("\nAverage daily forecast per service type:")
    for col in forecast_df.columns:
        avg_val = forecast_df[col].mean()
        print(f"  {col}: {avg_val:.1f}")

    print(f"\nTotal passengers forecasted over 7 days: {forecast_df.sum().sum():.1f}")

    # Step 9: Display model diagnostics
    print("\n" + "--"*50)
    print("Model Diagnostics")
    print("--"*50)
    print(f"Model type: VAR({results.k_ar})")
    print(f"Number of variables: {len(df_clean.columns)}")
    print(f"Training data period: {df_clean.index.min().strftime('%Y-%m-%d')} to {df_clean.index.max().strftime('%Y-%m-%d')}")
    print(f"Number of training observations: {results.nobs}")
    print(f"Forecast horizon: {forecast_horizon} days")

except FileNotFoundError:
    print("❌ File not found. Please verify the CSV file path and try again.")

except pd.errors.EmptyDataError:
    print("❌ The CSV file is empty. Please provide a valid dataset.")

except pd.errors.ParserError as pe:
    print(f"❌ CSV parsing error: {pe}")
    print("Please check the file's format and content.")

except ValueError as ve:
    print(f"❌ Data error: {ve}")

except Exception as ex:
    print(f"❌ Unexpected error ({type(ex).__name__}): {ex}")
    import traceback
    traceback.print_exc()

finally:
    warnings.resetwarnings()

print("\n" + "--"*50)
print("Forecasting and diagnostics successfully executed")
print("--"*50)


Loading dataset...
Dataset loaded with shape: (1919, 7)
Preview of the data:
         date  Local_Route  Light_Rail  Peak_Service  Rapid_Route  School  \
0        Date  Local Route  Light Rail  Peak Service  Rapid Route  School   
1  30/08/2024        16436       10705           225        19026    3925   
2  15/09/2023        15499       10671           267        18421    4519   
3  28/12/2021         1756        2352             0         3775       0   
4  11/01/2023        10536        8347           223        14072       0   

   Other  
0  Other  
1     59  
2     61  
3     13  
4     48  

Parsing the 'date' column...
Failed to parse with predefined formats, trying automatic parsing...

Converting data columns to numeric types...
Data types after conversion:
Local_Route     int64
Light_Rail      int64
Peak_Service    int64
Rapid_Route     int64
School          int64
dtype: object

Count of missing values per column:
Local_Route     0
Light_Rail      0
Peak_Service    0
Rapid_